In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

# Math behind the code
## for recurring simple deposit
### fv=(R+Rin) + (R+Ri(n-1))+ (R+Ri(n-2))+ .....+ (R+Ri1)
### fv=n*R+ iR ( n+(n-1)+(n-2)+.....+1)
### fv=n*R+ iR * n(n+1)/2
### fv= R {n+i* n(n+1)/2}
## if Simple FV=Compound FV
### R {n+i* n*(n+1)/2}=compound FV
### i = (FV/R-n)/ ((n*(n+1)/2))

In [2]:
# load the packages
import pandas as pd
import numpy as np

<IPython.core.display.Javascript object>

In [3]:
# set the parameters
tenure = [5, 10]
comp_int = [0.06, 0.065]
comp_freq = 1
installment = 25000
increment = 0.000001

<IPython.core.display.Javascript object>

In [4]:
def equivalent_rate(tenure, comp_int, comp_freq, installment, increment):
    future_val_comp = []
    future_val_simp = []
    equivalent_int = []
    for j in range(len(tenure)):

        # calculate the interest per payment (it varies based on frequency of payment and compounding freq)
        int_per_payment = (1 + comp_int[j] / comp_freq) ** (comp_freq / 12) - 1

        # get the compound interest rate future value
        future_val = np.fv(
            rate=int_per_payment, pv=0, pmt=-installment, nper=tenure[j] * 12, when=1
        )

        # get the list of fv for all the given compound int rate
        future_val_comp.append(future_val)

        # empty (almost) list of gap between simple fv and comp fv
        gap_list = [future_val]

        # get the first simple int rate to be tried with the set increment
        int_try = comp_int[j] + increment

        # get the fv for the above rate
        fvsx = installment * (
            (
                tenure[j] * 12
                + (int_try) / 12 * (tenure[j] * 12) * (tenure[j] * 12 + 1) / 2
            )
        )

        # calc the gap between comp fv and simple fv
        gap = np.absolute(fvsx - future_val)

        # append the list
        gap_list.append(gap)

        # as long as the gap is smaller for the simple rate fv than for the immediate previous one
        # increase the rate with increment and keep checking the gap
        while gap_list[-1] < gap_list[-2]:
            int_try = int_try + increment
            fvsx = installment * (
                (
                    tenure[j] * 12
                    + int_try / 12 * (tenure[j] * 12) * (tenure[j] * 12 + 1) / 2
                )
            )
            # get the gap for the current rate and append to the list
            gap = np.absolute(fvsx - future_val)
            gap_list.append(gap)

        # the last future value and int rate are the optimum fv and simple rate
        future_val_simp.append(fvsx)
        equivalent_int.append(int_try)

    # now get the summary table
    sum_tab = pd.DataFrame(
        {
            "Comp_Rate": comp_int,
            "Simple_Rate": equivalent_int,
            "Future_Val_Comp": future_val_comp,
            "Future_Val_Simp": future_val_simp,
        }
    )

    return sum_tab

<IPython.core.display.Javascript object>

In [5]:
equivalent_rate(tenure, comp_int, comp_freq, installment, increment)

,Comp_Rate,Simple_Rate,Future_Val_Comp,Future_Val_Simp
0,0.060,0.064421,1.745600e+06,1.745605e+06
1,0.065,0.078645,4.189490e+06,4.189506e+06


<IPython.core.display.Javascript object>